In [ ]:
def process(city):
    data = pd.read_excel('{} Walk score.xlsx'.format(city), engine='openpyxl')
    features_columns = [col for col in data.columns if col not in ['STREET_ID', 'WS']]
    min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0, 100))
    min_max_scaler = min_max_scaler.fit(data[features_columns])
    data_scaler = min_max_scaler.transform(data[features_columns])
    data_scaler = pd.DataFrame(data_scaler)
    data_scaler.columns = features_columns
    data_scaler['WS'] = data['WS']
    data_scaler=data_scaler.T
    return data_scaler

In [ ]:
def encrypt_all_workers_data(workers, g, h, n, K):
    encrypted_workers = np.zeros_like(workers, dtype=object)

    for i in range(workers.shape[1]):
        for j in range(workers.shape[0]):
            r = randint(1, n-1) 
            encrypted_workers[j, i] = encryption(workers[j, i], r, g, h, n, K)

    return encrypted_workers

In [ ]:
def encrypt_all_users_data(users, g, h, n, K):
    encrypted_users = np.zeros_like(users)
    encrypted_users[0, :] = users[0, :]  # User IDs are not encrypted

    for i in range(users.shape[1]):
        r = np.random.randint(1, n - 1)
        for j in range(1, users.shape[0]):
            encrypted_users[j, i] = encryption(users[j, i], r, g, h, n, K)

    return encrypted_users

In [ ]:
def calculate_all_ps_scores(workers):
    ps_scores = np.zeros(workers.shape[1])
    for i in range(workers.shape[1]):
        worker_i_data = workers[6:10, i]  
        workers_data = [np.column_stack((workers[j], workers[j+1])) for j in range(6, 10, 2)]
        ps_scores[i] = calculate_ps_score(worker_i_data, workers_data)
    workers[1] = ps_scores
    return workers

In [ ]:
def evaluate_worker_trust(workers):
    trusted_workers = []
    untrusted_workers = []
    
    for i in range(workers.shape[1]):
        worker_data = workers[6:10, i]
        threshold_values = np.zeros(4)
        if any((worker_data == x).all() for x in trusted_workers):
            continue
        
        for j in range(4):
            if len(trusted_workers) > 1:
                threshold_values[j] = np.mean([trusted_worker[j] for trusted_worker in trusted_workers if trusted_worker[j] != np.min(trusted_workers) and trusted_worker[j] != np.max(trusted_workers)])
            else:
                threshold_values[j] = worker_data[j]
        
        d = np.mean(np.abs(worker_data - threshold_values))
        level_num = 9
        
        if d <= 10:
            workers[2, i] = 1
        elif 11 <= d <= 5 * level_num:
            workers[2, i] = 0.5 * math.ceil(d / 5)
        else:
            workers[2, i] = 0.5 * level_num + 0.5

        if workers[2, i] <= 3:
            trusted_workers.append(worker_data)
        else:
            untrusted_workers.append(worker_data)

    return workers

In [ ]:
def evaluate_user_trust(users):
    trusted_users = []
    untrusted_users = []
        
    for i in range(users.shape[1]):
        user_data = users[2:6, i]
        threshold_values = np.zeros(4)
        
        if any((user_data == x).all() for x in trusted_users):
            continue
        
        for j in range(4):
            if len(trusted_users) > 1:
                threshold_values[j] = np.mean([trusted_user[j] for trusted_user in trusted_users if trusted_user[j] != np.min(trusted_users) and trusted_user[j] != np.max(trusted_users)])
            else:
                threshold_values[j] = user_data[j]
        
        d = np.mean(np.abs(user_data - threshold_values))
        level_num = 9
        
        if d <= 10:
            users[1, i] = 1
        elif 11 <= d <= 5 * level_num:
            users[1, i] = 0.5 * math.ceil(d / 5)
        else:
            users[1, i] = 0.5 * level_num + 0.5

        if users[1, i] <= 3:
            trusted_users.append(user_data)
        else:
            untrusted_users.append(user_data)

    return users

In [ ]:
def evaluate_users(users, workers, skyline):
    for i in range(skyline.shape[1]):
        t = np.where(workers[0, :] == skyline[0, i])[0][0]

        if workers[2, t] <= 3:
            if np.mean(users[2:6, i]) <= 70:
                users[1, i] += 0.5
        else:
            if np.mean(users[2:6, i]) > 70:
                users[1, i] += 0.5

    return users

In [ ]:
def select_skyline_workers(candidate_workers, n_skyline, t1, t2, t3, t4):
    num_workers = candidate_workers.shape[1]
    skyline_indices = []  
    ps_min = 0 
    
    for i in range(num_workers):  
        worker = candidate_workers[:, i]  
        if worker[2] <= t1 and worker[5] <= t2 and worker[3] >= t3 and worker[4] >= t4:  
            if len(skyline_indices) == 0:  
                skyline_indices.append(i)
                ps_min = worker[1]  
            elif worker[1] > ps_min and len(skyline_indices) < n_skyline:  
                skyline_indices.append(i)
                ps_min = min(candidate_workers[1, skyline_indices])  

  
    skyline_indices = np.array(skyline_indices)
    sorted_indices = np.argsort(candidate_workers[1, skyline_indices])[::-1]
    sorted_indices = sorted_indices[:n_skyline]

    return candidate_workers[:, skyline_indices[sorted_indices]]